# Personal Search 02

Busca de informações em livros usando FAISS

## Import packages

In [1]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_community.llms import Ollama
from IPython.display import display, Markdown
import logging

#logging.basicConfig(level=logging.INFO)

## Configuration

In [2]:
#model = "llama3:8b"
model = "mistral:7b"
embeddings_model = "nomic-embed-text:v1.5"
path_books = "./vector-store/books"
question = "Dificuldades na vida do Cristão"

## Retriever Instance

In [21]:
embeddings = OllamaEmbeddings(model=embeddings_model)

db = FAISS.load_local(path_books, embeddings, allow_dangerous_deserialization=True)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = False
retriever.search_kwargs["k"] = 2

## Search

In [22]:
sources = retriever.invoke(question)
livros = ''
for source in sources:
    source_name = source.metadata['source'].split('/')[-1]
    source_page = source.metadata['page']
    livros += f'**Fonte:** {source_name} - Página: {source_page}\n\n'
    livros += f'{source.page_content}\n\n\n'

In [23]:
# RAG
template = """
Cite partes do livro abaixo caso tenham relação com "{question}".

Livros:
{context}

Responda em português.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = Ollama(model=model, temperature=0)
#llm = ChatOllama(model=model, temperature=0)

final_rag_chain = (
    {"context": itemgetter("livros"), 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_answer = final_rag_chain.invoke({"question":question, "livros":livros})

In [24]:
display(Markdown(f"**Busca:** {question}\n\n"))
display(Markdown(final_answer))

**Busca:** Dificuldades na vida do Cristão



 As passagens do livro "Evangelho Maltrapilho" de Brennan Manning que podem se relacionar com "Dificuldades na vida do Cristão" são as seguintes:

1. Página 86 e 87: Neste trecho, o autor descreve a maneira como os cristãos podem se distanciar de Deus, ficando presos em uma rotina monótona e superficial, sem ver a graça divina em sua vida. Ele também escreve sobre o perdão que Deus oferece aos pecadores, independentemente de suas falhas passadas.
2. Página 87: Neste trecho, o autor conta a história do filho pródigo e descreve como ele retorna a casa de seu pai apenas para sobreviver, sem ver a graça e o amor que este lhe oferece. O autor também descreve a reação do pai ao ver o filho, que é marcada por compaixão e amor incondicional.

Em resumo, essas passagens discutem sobre como os cristãos podem se distanciar de Deus e perder a graça em suas vidas, além da importância do perdão divino e do amor infinito de Deus para nós, independentemente de nossas falhas.

In [28]:
sources = retriever.invoke(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source.metadata}"))
    #display(Markdown(f"**Trecho:** {source.page_content}"))

**Fonte:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 86}

**Fonte:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 87}

In [26]:
sources = db.similarity_search_with_score(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source[0].page_content}"))
    display(Markdown(f"**Trecho:** {source[0].metadata}"))
    display(Markdown(f"**Score:** {source[1]}"))

**Fonte:** ~ 87 ~ o outro lado, fechamos Deus para a nossa consci ência, nosso coração se  esfria. Os cristãos 
agnósticos não negam um Deus pessoal; eles demonstram a sua descrença quando ignoram o sagrado. A trivialidade da nossa vida é mudo testemunho da surrada mobília de nossa alma. 
Nossos dias vão assim se tornan do cada vez mais triviais. "Ficamos presos num labirinto 
frenético. Levantando quando o relógio determ ina. Bombardeados por manchetes de jornal 
que parecem remotas e além do alcance. Extenu ados por todas as oper ações mecânicas que nos 
lançam à atividade e à produtividade. Testados  pelo tráfego, forçados a calcular tempo e 
distância a nível de segundo. Elevadores, tel efones e engenhocas guiam-nos pelas interações 
necessárias e mantêm as interações humana s superficiais e num nível mínimo. Nossa 
concentração é interrompida por reuniões e pequen as crises. No fim do dia, rebobinamos a nós 
mesmos: tráfego, automação, manchetes, até que o alarme do relógio imponha a acordar de 
amanhã. Rotinas de procedimentos e de timing. Pouco espaço para responder humanamente e 
com humanidade aos eventos diários; pouco tempo para adentrar a sabedoria e o vigor e a 
promessa de suas oportunidades. Sentimos nossa vida sufocando-nos, confinando-nos e 
moldando-nos".
89 
Estabelecemos e nos conformamos a vidas de  piedade confortável e de virtude bem-
alimentada. Tornamo-nos complacentes e vivemos vidas práticas. Nossas débeis tentativas de 
orar são repletas de frases pomposas direci onadas a uma divindade impassível. Até mesmo 
ocasiões de adoração tornam-se triviais. 
E este o manquejar vitorioso freqüentemente  vivido por este escritor. Em momentos 
diferentes da jornada tentei encher o vazio qu e acompanha muitas veze s a presença de Deus 
através de uma variedade de substitutos: escrever , pregar, viajar, televisão, cinema, sorvete, 
relacionamentos superficiais, esportes, música, devaneio, álcool, etc. Como diz Annie Dillard: 
"Há sempre uma enorme tentação de se embrom ar fazendo amigos efêm eros, refeições efêmeras 
e viagens efêmeras ao longo de anos efêmeros sem fim".90 Ao longo do caminho optei pela 
escravidão e perdi todo o desejo de liberdade.  Amei meu cativeiro e aprisionei a mim mesmo 
no desejo por coisas que eu odiava. Endureci meu amor contra o amor verdadeiro. Abandonei 
a oração e fugi da simples santidade da minha vida. Num determinado dia, quando a graça me

**Trecho:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 86}

**Score:** 293.3338317871094

**Fonte:** ~ 88 ~ Deus sabe como perdoar e colocar todos os qu atro juntos. O pai do pródigo verdadeiramente 
disse: "Shh, menino. Não preciso saber onde  você esteve ou o que tem aprontado". 
O evangelho da graça anuncia: o perdão pre cede o arrependimento. O pecador é aceito 
antes de implorar por misericórdia. Ela já é as segurada. Ele precisa apenas aceitá-la. Anistia 
total. Perdão gracioso. "Só Deus é capaz de tornar  o perdão algo glorioso de se lembrar. Ele tem 
tanto prazer em nos perdoar que aqueles que lh e proporcionaram essa alegria não se sentem 
como pestinhas repulsivos e importunos, ma s como crianças mimadas, compreendidas e 
encorajadas, agradáveis e úteis para ele, infinitamente melhores  do que se consideravam. 'Que 
erro feliz!', eles poderiam gritar. Se não fôsse mos pecadores e não precisássemos do perdão mais 
do que de pão, não teríamos como saber quão profundo é o amor de Deus".91 
Quando o filho pródigo coxeou até sua casa de pois de sua prolongada farra de devassidão e 
vadiagem, bebedeira e promiscuidade, suas motivaç ões eram, na melhor das hipóteses, incertas. 
Ele disse a si mesmo: "Quantos trabalhadores de  meu pai têm abundância de pão, e eu aqui 
pereço de tome! Levantar-me-ei, e irei  ter com meu pai" (Lc 15:17,18; ARC).  O estômago do 
maltrapilho não estava doendo de remorso porque ele havia pa rtido o coração do pai. Ele 
cambaleou para casa simplesmente para sobrev iver. Sua permanência numa terra distante o 
havia deixado falido. Os dias de vinho e rosa s o haviam deixado atordoado e desiludido. O 
vinho azedou e as rosas murcharam. Sua decl aração de independência havia ceifado uma 
colheita inesperada: não liberdade,  alegria e uma vida nova, mas ca tiveiro, tristeza e um embate 
com a morte. Os amigos-da-onça haviam transfer ido suas lealdades quando o seu cofrinho se 
esvaziou. Desencantado com a vi da, o gastador traçou o cami nho de volta para casa, não 
ardendo de desejo de ver seu pai,  mas de apenas permanecer vivo. 
Para mim, o versículo mais tocante da Bíblia inteira é a reação do pai: "E, quando ainda 
estava longe, viu-o seu pai, e se moveu de íntima compaixão, e, correndo, lançou-se-lhe ao 
pescoço, e o beijou" (Lc 15:20; ARC).  Emociona-me que o pai não tenha submetido o rapaz a 
interrogatório cruzado, que não o tenha intimi dado, que não tenha dado um sermão sobre 
ingratidão, que não tenha insistid o em qualquer motivação superior. Ele ficou tão alegre ao ver

**Trecho:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 87}

...

**Trecho:** {'source': './books/import/O Peregrino - John Bunyan.pdf', 'page': 52}

**Score:** 304.92279052734375

**Fonte:** vida, o poder dos nossos cultos públicos e da devoção particular evapora rapidamente; oramos
como num sonho; louvamos e pregamos como sonâmbulos. Que Deus venha a nos acordar e
avivar, enviando a nova chuva matutina para renovar as forças de sua herança cansada.
Temos atualmente poucos gigantes cheios de graça que levantam a cabeça e os ombros acima da
estatura comum; homens que nos guiam em atos de heroísmo e demonstrações de fé inabalável.
Afinal de contas, a obra da igreja cristã, embora devesse ser feita por todos, é geralmente feita
por algumas pessoas que têm uma notável graça. Nesta era degenerada, somos muito parecidos
com Israel na época dos Juízes, pois há em nosso meio líderes que julgam Israel e são o terror
dos inimigos. Ah! Se a igreja ainda tivesse entre os seus membros uma raça de heróis! Se nossas
operações  missionárias  fossem  realizadas  com  a  bravura  santa  que  marcou  a  igreja  dos
primórdios; se pudéssemos trazer de volta os apóstolos e os mártires, ou mesmo Carey e Judson,
que milagres  seriam feitos! Temos  produzido uma raça  de duendes  e, de modo geral,  nos
contentamos com isso.
Existia um clube de baixinhos em Londres, cuja qualificação para aceitação no rol de membros
era não ter mais de um metro e meio de altura. Aqueles anões tinham a convicção — ou fingiam
ter — de que estavam mais perto da perfeição de masculinidade do que os outros, argumentando
que os homens primitivos tinham sido muito mais gigantescos do que os espécimes atuais, e,
consequentemente, o caminho da evolução era crescer cada vez menos. Sendo assim, à medida
que a espécie humana fosse se aperfeiçoando, os indivíduos se tornariam tão diminutos quanto
eles. É possível criar um clube de cristãos semelhante em Londres. Sem nenhuma dificuldade,
pode-se atingir um enorme número de membros, pois é muito comum a ideia de que nosso
cristianismo  anão é, afinal, o padrão, e muitos  até acham  que os cristãos mais  nobres são
espalhafatosos, fanáticos e de sangue quente; enquanto nós somos contidos porque sábios e
indiferentes porque inteligentes. Devemos acabar com essa bobagem. O fato é que a maioria de
nós é inferior aos cristãos primitivos, que foram perseguidos por serem totalmente cristãos. Não
sofremos  perseguição  porque  dificilmente  somos  plenamente  cristãos.  Eles  eram  tão
determinados na propagação do reino do Redentor que se tornaram o estorvo da época em que

**Trecho:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 23}

**Score:** 308.45294189453125